# ぴよログデータ分析

In [ ]:
import pandas as pd
import numpy as np
import re
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import os

In [ ]:
# 誕生日
birth_date = datetime.datetime(2020, 1, 1)

## データ読込

In [ ]:
# データ読込
path = './data'
files = os.listdir(path)
month_texts = []
for filename in files:
    f = open(f'{path}/{filename}', encoding='utf_8')
    data = f.read()
    month_texts.append(data)
    f.close()

In [ ]:
# 対象項目
def check_item(text):
    if re.findall('起きる|寝る|母乳|ミルク|離乳食|搾母乳', text) and re.match(r'([01][0-9]|2[0-3]):[0-5][0-9]', text):
        return True
    return False

In [ ]:
# 対象データをリスト化
def get_piyolog_all_items(month_texts):
    all_items = []

    for month_text in month_texts:
        # 改行で分割
        lines = month_text.splitlines()
        array = np.array(lines)

        for index, item in enumerate(array):
            # 日付取得
            if item == '----------' and index < len(array) - 1:
                day = array[index + 1][:-3]
                day_date = datetime.datetime.strptime(day, '%Y/%m/%d')

            # 対象項目の場合
            if item != '' and check_item(item):
                # 空白で分割
                record = item.split()

                record_dt = datetime.datetime.strptime(day + ' ' + record[0], '%Y/%m/%d %H:%M')
                record_type = None
                record_subtype = record[1]
                record_value = None
                record_timespan = None

                if '寝る' in record_subtype:
                    record_type = '睡眠'

                if '起きる' in record_subtype:
                    record_type = '睡眠'

                if 'ミルク' in record_subtype or '搾母乳' in record_subtype:
                    record_type = '食事'
                    # 搾母乳も項目名はミルクにする
                    record_subtype = 'ミルク'
                    # 時間は10分固定にする
                    record_timespan = 10
                    # ミルク量
                    record_value = int(record[2].replace('ml', ''))

                if '母乳' in record_subtype:
                    record_type = '食事'
                    record_time = 0
                    # 授乳時間の合計を計算する
                    for r in record[2:]:
                        if '分' in r:
                            record_time += int(r.replace('分',''))
                    record_timespan = record_time
                            
                # 記録
                all_items.append([day_date, record_dt, record_type, record_subtype, record_timespan, record_value])

    return all_items

In [ ]:
df = pd.DataFrame(get_piyolog_all_items(month_texts),columns=['日付', '日時','分類','項目','時間','ミルク量'])

In [ ]:
df.head()

## データ加工

In [ ]:
# 睡眠時間
s_sleep_timespan = df.query('分類=="睡眠"').sort_values('日時', ascending=False).日時.diff() * -1
s_sleep_timespan.name = '睡眠時間'
df = pd.concat([df, s_sleep_timespan], axis=1)

In [ ]:
# 睡眠時間(timedelta)を分に直す
def sleeptime_to_minutes(x):
    return x.睡眠時間.seconds / 60
df["睡眠時間"] = df.apply(lambda x:sleeptime_to_minutes(x),axis=1)

In [ ]:
# 時間帯
def replace_time_zone(x):
    # 7:00～19:00の間に寝たものは昼寝とする
    if x.日時.hour >= 7 and x.日時.hour <= 18:
        return '昼'
    else:
        return '夜'
        
df['時間帯'] = df.apply(lambda x:replace_time_zone(x),axis=1)

In [ ]:
# 月齢
month_old_list = []
for i in range(0,10):
    month_old_list.append([birth_date + relativedelta(months=i+1),i])
    
def replace_month_old(x):
    for month_old in month_old_list:
        if x.日付 < month_old[0]:
            return month_old[1]
        
df['月齢'] = df.apply(lambda x:replace_month_old(x),axis=1)

In [ ]:
df.head()

## 1日単位の集計

In [ ]:
# 1日の夜に起きている回数・時間
df_night_wakeup = df.query('項目=="起きる" and 時間帯=="夜"').groupby('日付').agg({'日時':'count', '睡眠時間':'sum'}).reset_index()
df_night_wakeup.columns = ['日付','夜に起きる回数','夜に起きている時間']

In [ ]:
# 1日の昼寝回数
df_nap = df.query('項目=="寝る" and 時間帯=="昼"').groupby('日付').agg({'日時':'count', '睡眠時間':'sum'}).reset_index()
df_nap.columns = ['日付','昼寝回数', '昼寝時間']

In [ ]:
# 1日の食事回数
df_milk = df.query('項目=="ミルク"').groupby('日付').agg({'日時':'count', 'ミルク量':'sum'}).reset_index()
df_milk.columns = ['日付','ミルク回数', 'ミルク量']
df_mother_milk = df.query('項目=="母乳"').groupby('日付').agg({'日時':'count', '時間':'sum'}).reset_index()
df_mother_milk.columns = ['日付','授乳回数', '授乳時間']
df_eat = df.query('項目=="離乳食"').groupby('日付').agg({'日時':'count', '時間':'sum'}).reset_index()
df_eat.columns = ['日付','離乳食回数', '離乳食時間']

In [ ]:
# 1日集計のdf結合
df_groupby_day = pd.merge(df_nap, df_night_wakeup, on='日付', how='left')
df_groupby_day = pd.merge(df_groupby_day, df_milk, on='日付', how='left')
df_groupby_day = pd.merge(df_groupby_day, df_mother_milk, on='日付', how='left')
df_groupby_day = pd.merge(df_groupby_day, df_eat, on='日付', how='left')

In [ ]:
# 月齢列の追加
df_groupby_day['月齢'] = df_groupby_day.apply(lambda x:replace_month_old(x),axis=1)

In [ ]:
df_groupby_day.head()

## 月齢別の集計

In [ ]:
# 夜寝の1回あたりの睡眠時間
df_night_sleep_per_event = df.query('項目=="寝る" and 時間帯=="夜"').groupby('月齢').agg({'睡眠時間':['mean','std']})
# 昼寝の1回あたりの睡眠時間
df_nap_per_event = df.query('項目=="寝る" and 時間帯=="昼"').groupby('月齢').agg({'睡眠時間':['mean','std']})
# 授乳の1回あたりの時間
df_mother_milk_per_event = df.query('項目=="母乳"').groupby('月齢').agg({'時間':['mean','std']})
# ミルクの1回あたりの量
df_milk_per_event = df.query('項目=="ミルク"').groupby('月齢').agg({'ミルク量':['mean','std']})


In [ ]:
df_night_sleep_per_event.columns = pd.MultiIndex.from_arrays([['1回あたりの夜寝時間','1回あたりの夜寝時間'],['mean','std']])
df_nap_per_event.columns = pd.MultiIndex.from_arrays([['1回あたりの昼寝時間','1回あたりの昼寝時間'],['mean','std']])
df_mother_milk_per_event.columns = pd.MultiIndex.from_arrays([['1回あたりの授乳時間','1回あたりの授乳時間'],['mean','std']])
df_milk_per_event.columns = pd.MultiIndex.from_arrays([['1回あたりのミルク量','1回あたりのミルク量'],['mean','std']])

In [ ]:
# 1日集計のデータを月齢別に集計
df_groupby_month = df_groupby_day.groupby('月齢').agg({'昼寝回数':['mean','std'], '昼寝時間':['mean','std'], '夜に起きる回数':['mean','std'], '夜に起きている時間':['mean','std'], 'ミルク回数':['mean','std'], 'ミルク量':['mean','std'], '授乳回数':['mean','std'], '授乳時間':['mean','std'], '離乳食回数':['mean','std']})

In [ ]:
df_groupby_month = pd.merge(df_groupby_month, df_night_sleep_per_event, on='月齢', how='left')
df_groupby_month = pd.merge(df_groupby_month, df_nap_per_event, on='月齢', how='left')
df_groupby_month = pd.merge(df_groupby_month, df_mother_milk_per_event, on='月齢', how='left')
df_groupby_month = pd.merge(df_groupby_month, df_milk_per_event, on='月齢', how='left')

In [ ]:
pd.options.display.precision = 1
df_groupby_month.head()

## 可視化

In [ ]:
pd.plotting.register_matplotlib_converters()
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
# 夜に起きる回数とお昼寝回数の月齢推移
df_graph_night_wakeup_count = df_groupby_month['夜に起きる回数']['mean']
df_graph_nap_count = df_groupby_month['昼寝回数']['mean']
df_graph_night_wakeup_nap_count = pd.concat([df_graph_night_wakeup_count, df_graph_nap_count], axis=1)
df_graph_night_wakeup_nap_count.columns = ['夜に起きる回数','昼寝回数']

In [ ]:
# 1回あたりの睡眠時間平均の月齢推移
df_graph_night_sleep_time_per_event = df_groupby_month['1回あたりの夜寝時間']['mean'] / 60
df_graph_nap_time_per_event = df_groupby_month['1回あたりの昼寝時間']['mean'] / 60
df_graph_night_sleep_nap_time_per_event = pd.concat([df_graph_night_sleep_time_per_event, df_graph_nap_time_per_event], axis=1)
df_graph_night_sleep_nap_time_per_event.columns = ['1回あたりの夜寝時間','1回あたりの昼寝時間']

In [ ]:
# 起きている時間/昼寝の月齢推移
df_graph_night_wakeup_time = df_groupby_month['夜に起きている時間']['mean'] / 60
df_graph_nap_time = df_groupby_month['昼寝時間']['mean'] / 60
df_graph_night_wakeup_nap_time = pd.concat([df_graph_night_wakeup_time, df_graph_nap_time], axis=1)
df_graph_night_wakeup_nap_time.columns = ['夜に起きている時間','昼寝時間']

In [ ]:
# 食事回数の月齢推移
df_graph_milk = df_groupby_month['ミルク回数']['mean']
df_graph_mother_milk = df_groupby_month['授乳回数']['mean']
df_graph_eat = df_groupby_month['離乳食回数']['mean']
df_graph_eat_count = pd.concat([df_graph_milk, df_graph_mother_milk], axis=1)
df_graph_eat_count = pd.concat([df_graph_eat_count, df_graph_eat], axis=1)
df_graph_eat_count.columns = ['ミルク','授乳','離乳食']

In [ ]:
# 1回あたりの授乳時間平均の月齢推移
df_graph_mother_milk_per_event = df_groupby_month['1回あたりの授乳時間']['mean']
# 1回あたりのミルク量平均の月齢推移
df_graph_milk_ml_per_event = df_groupby_month['1回あたりのミルク量']['mean']

In [ ]:
# ミルク量/授乳時間の月齢推移
df_graph_milk_ml = df_groupby_month['ミルク量']['mean']
df_graph_mother_milk_time = df_groupby_month['授乳時間']['mean'] / 60

In [ ]:
fig1,(ax1, ax2, ax3)=plt.subplots(1,3,figsize=(18,5))
ax1.set_title('夜に起きる回数とお昼寝回数の月齢推移')
sns.lineplot(data=df_graph_night_wakeup_nap_count, ax=ax1)
ax2.set_title('1回あたりの夜寝時間と昼寝時間の月齢推移')
sns.lineplot(data=df_graph_night_sleep_nap_time_per_event, ax=ax2)
ax3.set_title('夜に起きている時間と昼寝時間の月齢推移')
sns.lineplot(data=df_graph_night_wakeup_nap_time, ax=ax3)

fig2,(ax4, ax5, ax6)=plt.subplots(1,3,figsize=(18,5))
ax4.set_title('1日のミルク回数の月齢推移')
sns.lineplot(data=df_graph_milk, ax=ax4)
ax5.set_title('1回あたりのミルク量の月齢推移')
sns.lineplot(data=df_graph_milk_ml_per_event, ax=ax5)
ax6.set_title('1日のミルク量の月齢推移')
sns.lineplot(data=df_graph_milk_ml, ax=ax6)

fig3,(ax7,ax8,ax9)=plt.subplots(1,3,figsize=(18,5))
ax7.set_title('1日の授乳回数の月齢推移')
sns.lineplot(data=df_graph_mother_milk, ax=ax7)
ax8.set_title('1回あたりの授乳時間(分)の月齢推移')
sns.lineplot(data=df_graph_mother_milk_per_event, ax=ax8)
ax9.set_title('1日の授乳時間の月齢推移')
sns.lineplot(data=df_graph_mother_milk_time, ax=ax9)

■新生児期(0～1ヶ月)の特徴

夜に起きる回数が多く、1回あたりの夜寝時間が短く、夜に起きている時間トータルも長い。  
→夜間に何度も授乳(ミルク)が必要で親がまとまった睡眠がとれないため、夜はとても大変。

一方で昼寝回数も多く、1回あたりの昼寝時間は短い。昼寝時間トータルは長いため、起きている時間は短い。  
→昼寝時間トータルは長いが1回あたりの時間は短いため、睡眠不足で昼寝したくてもまとまって寝れないので体力的につらい。  
起きている時間は短くベビーベッドで寝ているだけなので、ある程度目は離せる。

食事の回数は多く、1回あたりのミルクの量は少ない。母乳の出が少ないため授乳時間は1回あたり・1日トータルともに長い。  
→ほぼ一日中ミルクと授乳をしていて、体力的にもつらい。

夜のつらさ　★★★★★★★  
昼のつらさ　☆★★★★

■ねんね期(2～4ヶ月)の特徴

夜に起きる回数がやや減り、1回あたりの夜寝時間すこし長くなり、夜に起きている時間トータルもやや短くなる。  
→新生児期よりはましだが、まだまだ夜間授乳が必要で睡眠不足が続く。

昼寝回数はまだ多く、1回あたりの昼寝時間は短い。昼寝時間トータルは長いため、起きている時間は短い。  
→睡眠不足だが多少は体力的にましになる。  
起きている時間は短くベビーベッドで寝ているだけなので、引き続きある程度目は離せる。

ミルクの回数はかなり減り、1回あたりのミルク量がかなり増える。授乳時間は1回あたり・1日トータルともに短くなる。  
→1回に飲める量が多くなるため、食事回数が減ってやや楽になる。  

夜のつらさ　☆★★★★  
昼のつらさ　☆☆☆★★

■寝返り・おすわり期(5～7ヶ月)

夜に起きる回数が減り、1回あたりの夜寝時間が長くなり、夜に起きている時間トータルも短くなる。  
→夜間授乳が減って親がある程度まとまった睡眠がとれるようになるため、夜はやや楽になる。

昼寝回数は減るが、1回あたりの昼寝時間はあまり変わらない。昼寝時間トータルは短くなり、起きている時間が長くなる。  
→睡眠不足が解消されるため、昼寝時間を自由時間として使えるようになる。  
起きている時間は長くなりあやす必要があるが、移動しないで勝手に遊んでくれるので多少は目が離せる。  
昼にリングフィットをやったりしていたので、結構余裕があった気がする。

1回あたりのミルク量がやや減り、1日トータルの量が減る。授乳はあまり変わらない。  
→離乳食がはじまるため、ミルク・授乳が減る。離乳食は準備と食べさせることに時間がかかるため、少し大変になる。  

夜のつらさ　☆☆★★★  
昼のつらさ　☆☆☆☆★

■はいはい期(8～10ヶ月)

夜に起きる回数がさらに減り、1回あたりの夜寝時間がさらに長くなり、夜に起きている時間トータルもさらに短くなる。  
→夜間授乳がかなり減り親がまとまった睡眠がとれるようになるため、夜はかなり楽になる。

昼寝回数はさらに減り、1回あたりの昼寝時間が長くなる。昼寝時間トータルは短くなり、起きている時間が長くなる。  
→昼寝時間が長くなるため、ある程度まとまった自由時間ができる。  
起きている時間はかなり長くなりあやす必要があり、はいはいで移動するので目が離せなくなってくる。  
子守にテレビが必須になってくる。Eテレが神になる。

ミルク・授乳の回数はさらに減る。  
→離乳食の回数が増えるため、ミルク・授乳ともにさらに減る。離乳食の回数が増えるとそこそこ忙しくなる。

夜のつらさ　☆☆☆☆★  
昼のつらさ　☆☆☆★★
